<a href="https://colab.research.google.com/github/eshal26/AI-Workout-Assisstant/blob/main/VGG_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import os
import shutil
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
train_dir = 'train_dataset'
val_dir = 'validation_dataset'
test_dir = 'test_dataset'

# Define transformations for training, validation, and testing data
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.8238, 0.8539, 0.9391], std=[0.1325, 0.1437, 0.0529])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.8238, 0.8539, 0.9391], std=[0.1325, 0.1437, 0.0529])
])

# Create datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_test_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=val_test_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:

from sklearn.decomposition import PCA

class CustomVGGWithPCA(nn.Module):
    def __init__(self, num_classes=2, pca_components=50):
        super(CustomVGGWithPCA, self).__init__()

        # Feature extraction layers
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 112x112x64

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 56x56x128
        )

        # Initialize PCA
        self.pca = PCA(n_components=pca_components)

        # Classifier layers
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(pca_components, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, num_classes)
        )

    def extract_features(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the features for PCA
        return x

    def forward(self, x):
        x = self.extract_features(x)
        device = x.device
        # Apply PCA if it is fitted
        if hasattr(self.pca, 'components_'):
            x = self.pca.transform(x.detach().cpu().numpy())  # Detach and convert to NumPy for PCA
            x = torch.from_numpy(x).to(device, dtype=torch.float32)   # Convert back to tensor on the orice
        else:
            raise RuntimeError("PCA must be fitted before the forward pass")

        x = self.classifier(x)
        return x


In [ ]:
from sklearn.decomposition import IncrementalPCA

def fit_pca_on_train_data(train_loader, model, batch_size=64):
    # Use IncrementalPCA for memory efficiency
    ipca = IncrementalPCA(n_components=model.pca.n_components, batch_size=batch_size)

    for images, _ in train_loader:
        images = images.to(device)
        with torch.no_grad():
            features = model.extract_features(images)
            features = features.detach().cpu().numpy()  # Ensure detach before numpy()
            ipca.partial_fit(features)  # Fit incrementally

    model.pca = ipca  # Assign the fitted IPCA back to model’s PCA attribute

# Example usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pca_components = 12
num_classes = 2

model = CustomVGGWithPCA(num_classes=num_classes, pca_components=pca_components).to(device)
fit_pca_on_train_data(train_loader, model)


In [ ]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct_predictions, total_samples = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = 100 * correct_predictions / total_samples
        print(f"Epoch {epoch+1}/{num_epochs}: Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

# Training call example
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=40)


Epoch 1/40: Training Loss: 2.1691, Accuracy: 76.88%
Epoch 2/40: Training Loss: 1.8396, Accuracy: 77.04%
Epoch 3/40: Training Loss: 2.0479, Accuracy: 76.88%
Epoch 4/40: Training Loss: 1.6761, Accuracy: 76.50%
Epoch 5/40: Training Loss: 1.6498, Accuracy: 77.04%
Epoch 6/40: Training Loss: 1.6048, Accuracy: 75.69%
Epoch 7/40: Training Loss: 1.4504, Accuracy: 77.34%
Epoch 8/40: Training Loss: 1.2089, Accuracy: 77.65%
Epoch 9/40: Training Loss: 1.1386, Accuracy: 77.53%
Epoch 10/40: Training Loss: 1.1732, Accuracy: 77.27%
Epoch 11/40: Training Loss: 1.0648, Accuracy: 77.73%
Epoch 12/40: Training Loss: 1.0000, Accuracy: 77.80%
Epoch 13/40: Training Loss: 0.9380, Accuracy: 77.73%
Epoch 14/40: Training Loss: 0.8669, Accuracy: 79.07%
Epoch 15/40: Training Loss: 0.8874, Accuracy: 77.42%
Epoch 16/40: Training Loss: 0.7937, Accuracy: 79.15%
Epoch 17/40: Training Loss: 0.8952, Accuracy: 78.26%
Epoch 18/40: Training Loss: 0.8247, Accuracy: 77.84%
Epoch 19/40: Training Loss: 0.8081, Accuracy: 77.53%
Ep